In [0]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from google.colab import drive
drive.mount('/content/drive')

In [0]:
df_activity = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/data/train_activity.csv', engine = 'python')
df_pledge = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/data/train_pledge.csv', engine = 'python')
df_payment = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/data/train_payment.csv', engine = 'python')
#df_trade = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/data/test2_trade.csv', engine = 'python')
df_combat = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/data/train_combat.csv', engine = 'python')

# ***TRAIN***

In [0]:
df1 = df_activity.copy()
server = pd.pivot_table(df1, index='acc_id',values='server',aggfunc=lambda x: x.mode().iat[0])
server =server.reset_index(level=0)
df1 =df1.drop(['char_id','server','day','exp_recovery','private_shop'],axis=1)
df1 = pd.pivot_table(df1, index='acc_id',aggfunc= np.sum).merge(server,on='acc_id')

# ***PLEDGE***

In [0]:
df2 = df_pledge.copy()
pledge_id = pd.pivot_table(df2, index='acc_id',values='pledge_id',aggfunc=lambda x: x.mode().iat[0])
pledge_id=pledge_id.reset_index(level=0)
pledge_ids, count = np.unique(pledge_id['pledge_id'],return_counts=True)
dict_hot_pledge_id = dict(zip(pledge_ids,count))
pledge_id = pd.concat([pledge_id,pledge_id['pledge_id'].map(dict_hot_pledge_id)],axis=1)
pledge_id.columns = ['acc_id','pledge_id','how_popular']
df2=df2.drop(['day','server','char_id','pledge_id'],axis=1)
df2 = pd.pivot_table(df2, index='acc_id',aggfunc= np.sum)
df2= df2.reset_index(level=0)
df2= df2.merge(pledge_id,on='acc_id')

# PAYMENT

In [0]:
df3 = df_payment.copy()
df3=df3.drop('day',axis=1)
df3 =pd.pivot_table(df3, index='acc_id',aggfunc= len)
df3 =df3.reset_index(level=0)
df3=df3.rename(columns={'amount_spent':'spent_count'})

# TRADE

In [0]:
#df4=df_trade.copy()

In [0]:
#df_trade.head()

In [0]:
#df4=df4.drop(['day','time','type','server','source_char_id','target_char_id','item_type','item_price','source_acc_id'],axis=1)

In [0]:
#df4 = df4.rename(columns = {'target_acc_id':'acc_id'})

In [0]:
#df4 = pd.pivot_table(df4, index='acc_id',aggfunc= len).merge(pd.pivot_table(df4, index='acc_id',aggfunc= np.sum),on='acc_id',how='inner')

In [0]:
#df4 =df4.reset_index(level=0)

In [0]:
#df4=df4.rename(columns = {'item_amount_x':'trade_count','item_amount_y': 'item_amount'})

# COMBAT

In [0]:
df5=df_combat.copy()
df5.head()
df5=df5.drop(['day','char_id','server'],axis=1)
df5_1=pd.pivot_table(df5, index='acc_id', values='class',aggfunc=lambda x: x.mode().iat[0])
df5_3=pd.pivot_table(df5, index='acc_id',values='level', aggfunc=max)
df5=df5.drop(['class','level'],axis=1)
df5_2 =pd.pivot_table(df5,index='acc_id', aggfunc=np.sum)
df5_1=df5_1.reset_index(level=0)
df5_2=df5_2.reset_index(level=0)
df5_3=df5_3.reset_index(level=0)
df5 = df5_1.merge(df5_2,on='acc_id')
df5 = df5.merge(df5_3,on = 'acc_id')
df5=df5.rename(columns={'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt':'combat_random_defender_cnt','same_pledge_cnt':'combat_same_pledge_cnt','temp_cnt':'combat_temp_cnt'
                   ,'etc_cnt':'combat_etc_cnt'})

In [0]:
df_final=df1.merge(df2,on='acc_id',how='outer')
df_final.shape

In [0]:
df_final=df_final.merge(df3,on='acc_id',how='outer')
df_final.shape

In [0]:
df_final=df_final.merge(df5,on='acc_id',how='outer')
df_final.shape

In [0]:
df_final=df_final.fillna(0)

In [0]:
df_final.describe()

In [0]:
temp_dict = {}
to_sort = df_final['server'].unique()
to_sort=np.sort(to_sort)
for a,b, in enumerate(to_sort):
  temp_dict[b]=a

df_final['server']=df_final['server'].map(temp_dict)
df_final = df_final.drop(['enchant_count'],axis=1)

In [0]:
####핫한 pledge 들 다구함
t1 = np.unique(df_final['pledge_id'],return_counts=True)

In [0]:
dic1 = {}
for i,j in zip(t1[0],t1[1]):
  dic1[i]=j
  
dic1[0]=0


In [0]:
df_final['pledge_id'] = df_final['pledge_id'].map(dic1)

In [0]:
df_final['fishing'] = df_final['fishing']/df_final['playtime']
df_final['party_exp_rate'] = df_final['party_exp']/df_final['playtime']
df_final['solo_exp_rate'] = df_final['solo_exp']/df_final['playtime']
df_final['revive_rate'] = df_final['revive']/df_final['death']


In [0]:
def death(x):
  if 0<=x<=0.5:
    return 0
  elif 0.5<x<=2.5:
    return 1
  else:
    return 2
df_final['death']=df_final['death'].apply(death)

In [0]:
def rich_monster(x):
  if 2<=x<8:
    return 1
  elif x>=8:
    return 2
  else:
    return 3
  
def level(x):
  if x==17:
    return 3
  elif x==16:
    return 2
  elif x<=12:
    return 0
  elif 12<x<16:
    return 1
  
df_final['rich_monster']=df_final['rich_monster'].apply(rich_monster)
df_final['level']=df_final['level'].apply(level)

In [0]:
df_final.shape

In [0]:
df_final.describe()

In [0]:
#df_final.to_csv('/content/drive/My Drive/Colab_Notebooks/final_csv/test2.csv',index=False)

In [0]:
df_label= pd.read_csv('/content/drive/My Drive/Colab_Notebooks/data/train_label.csv')

In [0]:
df_label.head()

In [0]:
df_train = df_final.merge(df_label,on='acc_id')

In [0]:
#df_train.to_csv('/content/drive/My Drive/Colab_Notebooks/data/df_train.csv',index=False)

In [0]:
df_train.shape

In [0]:
df_train1=df_train.copy()

In [0]:
df_train = df_train1.copy()

In [0]:
df_train = df_train.drop(['acc_id','combat_char_cnt','num_opponent','play_char_cnt','pledge_cnt','combat_play_time'], axis = 1)

# my 전처리.
#-------------------------------------------

In [0]:
from sklearn.model_selection import train_test_split
X=df_train.drop(['amount_spent', 'survival_time'], axis = 1)
y_spent=df_train['amount_spent']
#y_survival= df_train['survival_time']
X_train, X_test, y_train, y_test = train_test_split(X,y_spent,test_size=0.2,random_state = 2019)

In [0]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [0]:
#feature_fraction : 데이터의 paremater들 중에 각 round에 몇 %의 parameter를 random으로 골라 학습시킬지 정해주는 인자
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l1',
    #'objective':'mape', #이게 loss function 같은거고, metric이 valid할때 무슨 함수로 valid할지 정하는거임.
    #'metric': 'regression_l1', #l2가 제곱 loss, mape는 %로 오차 나타내줌
    #'metric': 'mae',
    'num_leaves':200,
    #'min_data':20,
    'min_data_in_leaf': 20, #20이 최적화라고 한다.
    'max_depth': 20,
    'max_bin':50,
    'learning_rate': 0.05,
    #'feature_fraction': 0.8,
    #'max_depth':20,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'num_trees':1000, #이게 train 반복횟수임
    #'verbose': 0,
    #'min_sum_hessian_in_leaf':11,
    'n_estimators' : 720,
    #'categorical_features' : ['maxServerOfAcc','main_server']
    #'max_cat_group':30
    
}
#l1 절대 l2 제곱

In [0]:
print('Start_Training')

gbm = lgb.train(params,
               lgb_train,
               valid_sets = lgb_eval,
               verbose_eval = 200,
               early_stopping_rounds = 100)

print()


In [0]:
#모델 survival_time_64  ## 여기서는 feature 하나도안떨궜음현재
from sklearn.externals import joblib
 
# save model
joblib.dump(gbm, '/content/drive/My Drive/Colab_Notebooks/amount_spent_model.pkl')
 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#####survival_bin_model####

In [0]:
df_train = df_train1.copy()

In [0]:
df_train.describe()

In [0]:
df_train = df_train.drop(['acc_id','combat_char_cnt','num_opponent','play_char_cnt','pledge_cnt','combat_play_time'], axis = 1)

# my 전처리.
#-------------------------------------------

In [0]:
#new_merged_all.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)
df_train.set_value(temp['survival_time']!=64,'bin_survival_time',0)
df_train.set_value(temp['survival_time']==64,'bin_survival_time',1)
# temp.drop(['acc_id'],axis=1,inplace=True)
a=0

In [0]:
from sklearn.model_selection import train_test_split
X=df_train.drop(['amount_spent', 'survival_time','bin_survival_time'], axis = 1)
#y_spent=df_train['amount_spent']
y_survival= df_train['bin_survival_time']
X_train, X_test, y_train, y_test = train_test_split(X,y_survival,test_size=0.2,random_state = 2019)

In [0]:
lgb_train = lgb.Dataset(X_train, label= y_train)
lgb_eval = lgb.Dataset(X_test, label=y_test)

In [0]:
X_test.shape

In [0]:
#feature_fraction : 데이터의 paremater들 중에 각 round에 몇 %의 parameter를 random으로 골라 학습시킬지 정해주는 인자
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    #'objective':'mape', #이게 loss function 같은거고, metric이 valid할때 무슨 함수로 valid할지 정하는거임.
    #'metric': 'regression_l1', #l2가 제곱 loss, mape는 %로 오차 나타내줌
    #'metric': 'mae',
    'num_leaves':200,
    #'min_data':20,
    'min_data_in_leaf': 20, #20이 최적화라고 한다.
    'max_depth': 20,
    'max_bin':50,
    'learning_rate': 0.05,
    #'feature_fraction': 0.8,
    #'max_depth':20,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'num_trees':1000, #이게 train 반복횟수임
    #'verbose': 0,
    #'min_sum_hessian_in_leaf':11,
    'n_estimators' : 720,
    #'categorical_features' : ['maxServerOfAcc','main_server']
    #'max_cat_group':30
    
}
#l1 절대 l2 제곱

In [0]:
print('Start_Training')

gbm = lgb.train(params,
               lgb_train,
               valid_sets = lgb_eval,
               verbose_eval = 10,
               early_stopping_rounds = 100)

print()


In [0]:
#Prediction
y_pred=gbm.predict(X_test)
print(y_pred[:5])


#convert into binary values
for i in range(0,y_pred.shape[0]):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0
        
print(y_pred[:5])

In [0]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)

print(cm)
print(accuracy)

In [0]:
#모델 survival_time_64  ## 여기서는 feature 하나도안떨궜음현재
from sklearn.externals import joblib
 
# save model
joblib.dump(gbm, '/content/drive/My Drive/Colab_Notebooks/bin_survival.pkl')
 

In [0]:
df_train.describe()

In [0]:
###### survival_time_model ###########

In [0]:
df_train = df_train1.copy()

In [0]:
df_train = df_train.drop(['acc_id','combat_char_cnt','num_opponent','play_char_cnt','pledge_cnt','combat_play_time'], axis = 1)

# my 전처리.
#-------------------------------------------

In [0]:
df_train = df_train[df_train['survival_time']<64]

In [0]:
df_train = df_train1.copy()

In [0]:
from sklearn.model_selection import train_test_split
X=df_train.drop(['amount_spent', 'survival_time'], axis = 1)
#y_spent=df_train['amount_spent']
y_survival= df_train['survival_time']
X_train, X_test, y_train, y_test = train_test_split(X,y_survival,test_size=0.1,random_state = 2019)

In [0]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [0]:
#feature_fraction : 데이터의 paremater들 중에 각 round에 몇 %의 parameter를 random으로 골라 학습시킬지 정해주는 인자
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l1',
    #'objective':'mape', #이게 loss function 같은거고, metric이 valid할때 무슨 함수로 valid할지 정하는거임.
    #'metric': 'regression_l1', #l2가 제곱 loss, mape는 %로 오차 나타내줌
    #'metric': 'mae',
    'num_leaves':200,
    #'min_data':20,
    'min_data_in_leaf': 20, #20이 최적화라고 한다.
    'max_depth': 20,
    'max_bin':50,
    'learning_rate': 0.05,
    #'feature_fraction': 0.8,
    #'max_depth':20,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'num_trees':1000, #이게 train 반복횟수임
    #'verbose': 0,
    #'min_sum_hessian_in_leaf':11,
    'n_estimators' : 720,
    #'categorical_features' : ['maxServerOfAcc','main_server']
    #'max_cat_group':30
    
}
#l1 절대 l2 제곱

In [0]:
print('Start_Training')

gbm = lgb.train(params,
               lgb_train,
               valid_sets = lgb_eval,
               verbose_eval = 200,
               early_stopping_rounds = 100)

print()


In [0]:
#모델 survival_under_time_64 
from sklearn.externals import joblib
 
# save model
joblib.dump(gbm, '/content/drive/My Drive/Colab_Notebooks/survival_model.pkl')
 

In [0]:
t1 = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/final_csv/test1.csv')

In [0]:
len(t1)